In [14]:
import os
from torch.utils.data import Dataset
from PIL import Image

class DefectDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform

        label_map = {
            "Non defective": 0,
            "Defective": 1
        }

        for class_name, label in label_map.items():
            class_dir = os.path.join(root_dir, class_name)

            for f in os.listdir(class_dir):
                if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(
                        os.path.join(class_dir, f)
                    )
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# ------------------------
# 1. Device (GPU/CPU)
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------
# 2. Data transforms
# ------------------------
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# ------------------------
# 3. Datasets and Dataloaders
# ------------------------

train_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Train",transform = train_transform)
test_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Test",transform = test_transform)



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# ------------------------
# 4. Load pretrained ResNet18
# ------------------------
model = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: defective/non_defective

model = model.to(device)

# ------------------------
# 5. Loss and optimizer
# ------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ------------------------
# 6. Training function
# ------------------------
def train_loop(dataloader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0:
            print(f"Batch {i}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader)
    print(f"Training Loss: {epoch_loss:.4f}")

# ------------------------
# 7. Testing function
# ------------------------
def test_loop(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # store for analysis
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100.0 * correct / total

    print(
        f"Test Loss: {epoch_loss:.4f}, "
        f"Accuracy: {accuracy:.2f}% "
        f"({correct}/{total})"
    )

    return epoch_loss, accuracy, all_preds, all_labels


# ------------------------
# 8. Train & Test
# ------------------------
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # Train
    train_loop(train_loader, model, criterion, optimizer, device)

    # Test / validation
    test_loss, test_acc, y_pred, y_true = test_loop(test_loader, model, criterion, device)

    # Optional: store history for plotting later
    # train_losses.append(train_loss)  # if your train_loop returns loss
    # test_losses.append(test_loss)
    # test_accuracies.append(test_acc)

print("Training complete!")


Using device: cuda

Epoch 1/10


C:\Users\ismai\anaconda3\envs\project_1_2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ismai\anaconda3\envs\project_1_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0, Loss: 0.6768
Training Loss: 0.6641
Test Loss: 0.6214, Accuracy: 67.86% (57/84)

Epoch 2/10
Batch 0, Loss: 0.5862
Training Loss: 0.6021
Test Loss: 0.6146, Accuracy: 70.24% (59/84)

Epoch 3/10
Batch 0, Loss: 0.5494
Training Loss: 0.5341
Test Loss: 0.5509, Accuracy: 75.00% (63/84)

Epoch 4/10
Batch 0, Loss: 0.5283
Training Loss: 0.4955
Test Loss: 0.5210, Accuracy: 75.00% (63/84)

Epoch 5/10
Batch 0, Loss: 0.3928
Training Loss: 0.4705
Test Loss: 0.5067, Accuracy: 70.24% (59/84)

Epoch 6/10
Batch 0, Loss: 0.3951
Training Loss: 0.4219
Test Loss: 0.5084, Accuracy: 76.19% (64/84)

Epoch 7/10
Batch 0, Loss: 0.3866
Training Loss: 0.3805
Test Loss: 0.5087, Accuracy: 78.57% (66/84)

Epoch 8/10
Batch 0, Loss: 0.3942
Training Loss: 0.3581
Test Loss: 0.4857, Accuracy: 80.95% (68/84)

Epoch 9/10
Batch 0, Loss: 0.3435
Training Loss: 0.3467
Test Loss: 0.3637, Accuracy: 85.71% (72/84)

Epoch 10/10
Batch 0, Loss: 0.3418
Training Loss: 0.3037
Test Loss: 0.4200, Accuracy: 83.33% (70/84)
Training co

In [20]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

[[38  4]
 [10 32]]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84        42
           1       0.89      0.76      0.82        42

    accuracy                           0.83        84
   macro avg       0.84      0.83      0.83        84
weighted avg       0.84      0.83      0.83        84



In [18]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


[[34  8]
 [16 26]]
              precision    recall  f1-score   support

           0       0.68      0.81      0.74        42
           1       0.76      0.62      0.68        42

    accuracy                           0.71        84
   macro avg       0.72      0.71      0.71        84
weighted avg       0.72      0.71      0.71        84



In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# ------------------------
# 1. Device (GPU/CPU)
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------
# 2. Data transforms
# ------------------------
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# ------------------------
# 3. Datasets and Dataloaders
# ------------------------

train_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Train",transform = train_transform)
test_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Test",transform = test_transform)



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# ------------------------
# 4. Load pretrained ResNet18
# ------------------------
model = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: defective/non_defective

model = model.to(device)

# ------------------------
# 5. Loss and optimizer
# ------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ------------------------
# 6. Training function
# ------------------------
def train_loop(dataloader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0:
            print(f"Batch {i}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader)
    print(f"Training Loss: {epoch_loss:.4f}")

# ------------------------
# 7. Testing function
# ------------------------
def test_loop(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total
    print(f"Test Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

# ------------------------
# 8. Train & Test
# ------------------------
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loop(train_loader, model, criterion, optimizer, device)
    test_loop(test_loader, model, criterion, device)

print("Training complete!")


Using device: cuda

Epoch 1/10
Batch 0, Loss: 0.7541
Training Loss: 0.7341
Test Loss: 0.6983, Accuracy: 54.55%

Epoch 2/10
Batch 0, Loss: 0.6764
Training Loss: 0.6637
Test Loss: 0.6882, Accuracy: 59.09%

Epoch 3/10
Batch 0, Loss: 0.6011
Training Loss: 0.5786
Test Loss: 0.6527, Accuracy: 63.64%

Epoch 4/10
Batch 0, Loss: 0.5235
Training Loss: 0.4956
Test Loss: 0.6588, Accuracy: 68.18%

Epoch 5/10
Batch 0, Loss: 0.4025
Training Loss: 0.4908
Test Loss: 0.5447, Accuracy: 68.18%

Epoch 6/10
Batch 0, Loss: 0.3940
Training Loss: 0.3993
Test Loss: 0.5378, Accuracy: 72.73%

Epoch 7/10
Batch 0, Loss: 0.3806
Training Loss: 0.4053
Test Loss: 0.4993, Accuracy: 72.73%

Epoch 8/10
Batch 0, Loss: 0.3720
Training Loss: 0.3663
Test Loss: 0.5442, Accuracy: 68.18%

Epoch 9/10
Batch 0, Loss: 0.2760
Training Loss: 0.3469
Test Loss: 0.5774, Accuracy: 81.82%

Epoch 10/10
Batch 0, Loss: 0.3785
Training Loss: 0.3107
Test Loss: 0.4129, Accuracy: 77.27%
Training complete!


In [13]:
for images, labels in test_loader:
    print(images.shape)
    print(labels.shape)
    break


torch.Size([22, 3, 224, 224])
torch.Size([22])


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# ------------------------
# 1. Device (GPU/CPU)
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------
# 2. Data transforms
# ------------------------
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# ------------------------
# 3. Datasets and Dataloaders
# ------------------------

train_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Train",transform = train_transform)
test_dataset = DefectDataset(r"C:\Users\ismai\Downloads\archive (15)\Railway Track fault Detection Updated\Test",transform = test_transform)



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# ------------------------
# 4. Load pretrained ResNet18
# ------------------------
model = models.resnet50(pretrained=True)
# Modify the final fully connected layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: defective/non_defective

model = model.to(device)

# ------------------------
# 5. Loss and optimizer
# ------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ------------------------
# 6. Training function
# ------------------------
def train_loop(dataloader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0:
            print(f"Batch {i}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader)
    print(f"Training Loss: {epoch_loss:.4f}")

# ------------------------
# 7. Testing function
# ------------------------
def test_loop(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # store for analysis
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100.0 * correct / total

    print(
        f"Test Loss: {epoch_loss:.4f}, "
        f"Accuracy: {accuracy:.2f}% "
        f"({correct}/{total})"
    )

    return epoch_loss, accuracy, all_preds, all_labels


# ------------------------
# 8. Train & Test
# ------------------------
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # Train
    train_loop(train_loader, model, criterion, optimizer, device)

    # Test / validation
    test_loss, test_acc, y_pred, y_true = test_loop(test_loader, model, criterion, device)

    # Optional: store history for plotting later
    # train_losses.append(train_loss)  # if your train_loop returns loss
    # test_losses.append(test_loss)
    # test_accuracies.append(test_acc)

print("Training complete!")


Using device: cuda


C:\Users\ismai\anaconda3\envs\project_1_2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ismai\anaconda3\envs\project_1_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/10
Batch 0, Loss: 0.7719
Training Loss: 0.7155
Test Loss: 0.6920, Accuracy: 46.43% (39/84)

Epoch 2/10
Batch 0, Loss: 0.6996
Training Loss: 0.6731
Test Loss: 0.6275, Accuracy: 70.24% (59/84)

Epoch 3/10
Batch 0, Loss: 0.6358
Training Loss: 0.5972
Test Loss: 0.5916, Accuracy: 65.48% (55/84)

Epoch 4/10
Batch 0, Loss: 0.5781
Training Loss: 0.5315
Test Loss: 0.5364, Accuracy: 73.81% (62/84)

Epoch 5/10
Batch 0, Loss: 0.5087
Training Loss: 0.4804
Test Loss: 0.5009, Accuracy: 76.19% (64/84)

Epoch 6/10
Batch 0, Loss: 0.4214
Training Loss: 0.4456
Test Loss: 0.5233, Accuracy: 75.00% (63/84)

Epoch 7/10
Batch 0, Loss: 0.4204
Training Loss: 0.4040
Test Loss: 0.4931, Accuracy: 80.95% (68/84)

Epoch 8/10
Batch 0, Loss: 0.3792
Training Loss: 0.3793
Test Loss: 0.4199, Accuracy: 83.33% (70/84)

Epoch 9/10
Batch 0, Loss: 0.3424
Training Loss: 0.3416
Test Loss: 0.4416, Accuracy: 82.14% (69/84)

Epoch 10/10
Batch 0, Loss: 0.3262
Training Loss: 0.2983
Test Loss: 0.4337, Accuracy: 80.95% (68/84)

In [22]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


[[37  5]
 [11 31]]
              precision    recall  f1-score   support

           0       0.77      0.88      0.82        42
           1       0.86      0.74      0.79        42

    accuracy                           0.81        84
   macro avg       0.82      0.81      0.81        84
weighted avg       0.82      0.81      0.81        84

